In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.utils import to_categorical

Using TensorFlow backend.


## 载入数据

In [2]:
import pickle

dataset = None

with open('../../common/xo_dataset.bin', 'rb') as f:
    dataset = pickle.load(f)

In [3]:
# 训练集图像向量
train_images = dataset['train_images']
# 训练集标签
train_labels = dataset['train_labels']
# 测试集图像向量
test_images = dataset['test_images']
# 测试集标签
test_labels = dataset['test_labels']

In [4]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [5]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## 构建模型

In [6]:
def NeuralNetwork(input_layer):
    flat = Flatten()(input_layer)
    dense1 = Dense(128, activation='relu')(flat)
    dense2 = Dense(128, activation='relu')(dense1)
    output = Dense(2, activation='softmax', name='output_node')(dense2)
    return output

In [7]:
# 定义输入层
input_layer = Input(shape=(28, 28, 1), name='input_node')

In [8]:
# call the model?
logits = NeuralNetwork(input_layer)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# 定义模型
keras_model = Model(input_layer, logits)

In [10]:
# 编译模型
keras_model.compile(optimizer = keras.optimizers.Adam(lr = 0.0001), \
    loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
keras_model.input_names

['input_node']

In [12]:
keras_model.output_names

['output_node']

## Keras模型转换为Estimator

https://www.tensorflow.org/guide/estimators


https://www.tensorflow.org/guide/estimators#creating_estimators_from_keras_models

采用 Estimator 进行编程的概览

https://www.tensorflow.org/guide/premade_estimators#overview_of_programming_with_estimators


In [13]:

#convert to an Estimator
# the model_dir states where the graph and checkpoint files will be saved to
est_model = tf.keras.estimator.model_to_estimator(keras_model = keras_model, model_dir = './Keras_Neural_Network')

# 模型文件graph.pbtxt 还有检查点会保存在相对路径 ./Keras_Neural_Network

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Using config: {'_model_dir': './Keras_Neural_Network', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f43315e2d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:

# 准备训练数据
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={keras_model.input_names[0]: train_images},
    y=train_labels,
    batch_size=128,
    num_epochs=20,
    shuffle=False)

est_model.train(input_fn = train_input_fn, steps=2000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='./Keras_Neural_Network/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('./Keras_Neural_Network/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: output_node/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: output

In [17]:
# 查看输入尺寸
keras_model.input_shape

(None, 28, 28, 1)

[搭配 Estimator 使用 SavedModel
](https://www.tensorflow.org/guide/saved_model#using_savedmodel_with_estimators)


准备提供输入

https://www.tensorflow.org/guide/saved_model#prepare_serving_inputs

`serving_input_receiver_fn()`

在投入使用系统将向其发出推理请求的图中添加占位符。
添加将数据从输入格式转换为模型所预期的特征 Tensor 所需的任何额外操作。

In [21]:

def serving_input_receiver_fn():
    '''用于接收推理请求'''
    # 占位符 制定输入的数据类型，尺寸
    # 问题？ 需要重命名么?
    input_node = tf.placeholder(dtype=tf.float32, shape=list(keras_model.input_shape), name='input_node')
    # 接受的张量
    receiver_tensors = {'input_node': input_node}
    # 问题，这里的Feature是做什么的
    features = receiver_tensors
    
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)


# 注：'saved_model'保存文件的路径，可以修改为其他　
est_model.export_saved_model('saved_model', serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from ./Keras_Neural_Network/model.ckpt-1501
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model/temp-b'1557321954'/saved_model.pb


b'saved_model/1557321954'


目录创建了一个文件夹叫做`saved_model`, 里面其中一个文件夹叫做`1557321954`
里面有pb文件, 还有变量文件.
```
src$ tree saved_model/
saved_model/
└── 1557321954
    ├── saved_model.pb
    └── variables
        ├── variables.data-00000-of-00001
        └── variables.index

2 directories, 3 files
```

##  Estimator转换成TFlite
[How to convert a Tensorflow Estimator to Tensorflow Lite](https://medium.com/datadriveninvestor/how-to-convert-a-tensorflow-estimator-to-tensorflow-lite-a3509a9ba902)

## Freeze_Graph冻结图


[面向工具开发者的 TensorFlow 模型文件指南](https://www.tensorflow.org/guide/extend/model_files#freezing)
在训练期间权重通常不会存储在文件格式内，而是保存在单独的检查点文件中，并且图中的 Variable 操作可在初始化操作时加载最新的值。在部署到生产环境时，使用单独的文件往往不是很方便，因此有一个 freeze_graph.py 脚本可以获取图定义和一组检查点，并将它们冻结到一个文件中。

这样做是为了加载 GraphDef，从最新的检查点文件中提取所有变量的值，然后将每个 Variable 操作替换为 Const（其中包含存储在其属性中的权重的数值数据）。然后，它会剥离所有未用于前向推断的无关节点，并将生成的 GraphDef 保存到输出文件中。

[脚本地址 freeze_graph.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py)


```
freeze_graph --input_checkpoint=model.ck
pt  --output_node_names=output_node
```


## Reference

[Freezing a Keras model-Joseph Bullock](https://towardsdatascience.com/freezing-a-keras-model-c2e26cb84a38)